In [27]:
import numpy as np

from keras.models import Model
from keras.layers import Input
from keras.layers import Flatten, Dense
from keras.layers.convolutional import Conv2D, Convolution2D

In [24]:
Conv2D()

In [28]:
Convolution2D()

TypeError: __init__() missing 3 required positional arguments: 'nb_filter', 'nb_row', and 'nb_col'